In [1]:
# ============== IMPORTANT - USE YoloV8 environment ====================

# ============== IMPORTANT - It may need Conda ====================

from ultralytics import YOLO
import os
from PIL import ImageDraw, ImageFont, Image
import numpy as np

# Load a pretrained YOLO model (recommended for training)
model = YOLO('models/best.pt')

# specify the path to the folder containing the images
folder_path = 'dataset/images/train'

# get a list of all the files in the folder
file_list = os.listdir(folder_path)

image_index = 8

# joint the folder path and the image name
image_path = os.path.join(folder_path, file_list[image_index])

# Run inference on a single image
results = model(image_path)

# get the label, confidence and xyxy from the results
for r in results:
    label = r.boxes.cls.cpu().numpy().astype(int)
    conf = r.boxes.conf.cpu().numpy().astype(float)
    xyxy = r.boxes.xyxy.cpu().numpy().astype(int)

    # joint the label, confidence and xyxy into a single list of one dimension
    label_conf_xyxy = np.concatenate((label.reshape(-1,1), conf.reshape(-1,1), xyxy), axis=1)

# # get p1 and p2
p1 = label_conf_xyxy[:,2:4]
p2 = label_conf_xyxy[:,4:6]

# convert p1 and p2 to a list of tuples adn keep them as integers
p1 = [tuple(map(int, p)) for p in p1]
p2 = [tuple(map(int, p)) for p in p2]

# # show the image with the bounding boxes using the tuples p1 and p2
img = Image.open(image_path)

# convert the image to RGB
img = img.convert('RGB')

# draw the bounding boxes on the image and show the image
for i in range(len(p1)):
    draw = ImageDraw.Draw(img)
    draw.rectangle((p1[i], p2[i]), outline='red', width=3)
img.show()




image 1/1 c:\Users\me1elar\Documents\GitHub\YoloV8\dataset\images\train\Part 1_A00762157_Res_1_Normals Y.png: 640x640 1 Pinsite, 6.0ms
Speed: 5.0ms preprocess, 6.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)


In [2]:
# aupdate yolov8.yaml file

import yaml

with open('yolov8.yaml') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
    print(data)
# update the names parameter
data['names'][1] = 'person'
data['names'][2] = 'bicycle'

# save the updated yaml file
with open('yolov8.yaml', 'w') as f:
    yaml.dump(data, f)


                 


{'path': 'C:\\Users\\me1elar\\Documents\\GitHub\\YoloV8\\dataset', 'train': 'images/train', 'val': 'images/val', 'test': None, 'names': {0: 'Pinsite'}}


In [1]:
from PIL import Image
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/train/weights/best.pt')

# joint the folder path and the image name
image_path = 'test_image.png'

# Run inference on a single image
results = model(image_path)

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image



image 1/1 c:\Users\me1elar\Documents\GitHub\YoloV8\static\images\Part 1_A00759575_Res_3_Normals X.png: 640x640 2 Pinsites, 19.8ms
Speed: 0.0ms preprocess, 19.8ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)


In [1]:
from ultralytics import YOLO
import os
from PIL import Image
import numpy as np
import cv2

# Load a pretrained YOLO model (recommended for training)
model = YOLO('runs/detect/train3/weights/best.pt')

# specify the path to the folder containing the test images
folder_path = 'dataset/images/val'

# get a list of all the files in the folder
file_list = os.listdir(folder_path)

image = file_list[2]

# Run inference on a single image
results = model.predict('dataset/images/val/'+image, save=False, show=False)

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    # type of im_array
    print(type(im_array))
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    # convert the PIL image to a numpy array
    im_array = np.array(im)
    # show image with openCV
    cv2.imshow('image', im_array)
    cv2.waitKey(0)



image 1/1 c:\Users\me1elar\Documents\GitHub\AI-23-19-AVI-DefectDetectionWithYoloV8\dataset\images\val\Part 1_A00758766_Res_1_Normals X.png: 640x640 1 Scale, 6.0ms
Speed: 5.0ms preprocess, 6.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)


<class 'numpy.ndarray'>


In [2]:
import os
folder_path = r"C:data\test"
# get the list of subfolders in the folder
subfolders = os.listdir(folder_path)

print(subfolders)


['MachineScar', 'Rx', 'Scale', 'Sliver']


Apply image equalization to the input image.

In [5]:
import matplotlib.pyplot as plt
from skimage import data, exposure, img_as_float, io, color, img_as_ubyte
import os

# get the path of files inside the folder data/Pinsite
path = 'dataset/images/val'
files = os.listdir(path)

# iterate through the files and apply the histogram equalization
for file in files:
    img = io.imread(os.path.join(path, file))
    # print(img.shape)
    img_eq = exposure.equalize_adapthist(img_as_float(img))
    # Convert the image to RGB mode
    img_rgb = color.gray2rgb(img_eq)
    # convert to uint8
    img_rgb = img_as_ubyte(img_rgb)
    # save the image
    io.imsave('dataset/images/val_equalized/' + file, img_rgb)
   

Replace the label zero to one for the previous pinsite label images

In [9]:
# Current txt files contain dat as follows:
# 0 0.6109375 0.08984375 0.05625 0.0734375
# 0 0.60859375 0.6328125 0.0546875 0.06875

# where the first column is the class. This values needs to be change to be 1
# the files are located in the folder dataset\labels_old\train

import os

# Set the directory path
dir_path = r'dataset\labels_old\train_temp'

# Loop through each file in the directory
for filename in os.listdir(dir_path):
    if filename.endswith('.txt'):
        # Open the file in read mode
        with open(os.path.join(dir_path, filename), 'r') as file:
            lines = file.readlines()

        # Loop through each line in the file
        for i in range(len(lines)):
            # Split the line into a list of strings
            line = lines[i].split()
            # Change the first element of the list from '0' to '1'
            line[0] = '1'
            # Join the list back into a string keeping the number of rows the same
            lines[i] = ' '.join(line) + '\n'            

        # Write the modified lines back to the file
        with open(os.path.join(dir_path, filename), 'w') as file:
            file.writelines(lines)

Visualize the 640x640 image with its corresponding normalized bounding boxes and labels

In [5]:
import os
import cv2

# Specify the path to the folder containing the images
image_folder_path = 'dataset/images/train'

# Specify the path to the folder containing the txt files
txt_folder_path = 'dataset/labels/train'

# Get a list of all the txt files in the folder
txt_files = [file for file in os.listdir(txt_folder_path) if file.endswith('.txt')]

# Initialize a counter
counter = 0

# Define a dictionary that maps label integers to their corresponding names
label_names = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'D'}

# Loop through each txt file
for txt_file in txt_files:
    # Increment the counter
    counter += 1

    # If the counter is greater than 5, break the loop
    if counter > 5:
        break

    # Get the corresponding image file name
    image_file_name = os.path.splitext(txt_file)[0] + '.png'
    
    # Join the image folder path and the image file name
    image_path = os.path.join(image_folder_path, image_file_name)
    
    # Read the image using OpenCV
    image = cv2.imread(image_path)
    
    # Get the path to the txt file
    txt_file_path = os.path.join(txt_folder_path, txt_file)
    
    # Open the txt file in read mode
    with open(txt_file_path, 'r') as file:
        lines = file.readlines()
    
    # Loop through each line in the txt file
    for line in lines:
        # Split the line into a list of values
        values = line.split()
        
        # Extract the label
        label = int(values[0])
        
        # Get the label name from the dictionary
        label_name = label_names[label]
        
        # Extract the bounding box coordinates
        x, y, width, height = map(float, values[1:])
        
        # Calculate the top-left and bottom-right coordinates of the bounding box
        x1 = int((x - width / 2) * image.shape[1])
        y1 = int((y - height / 2) * image.shape[0])
        x2 = int((x + width / 2) * image.shape[1])
        y2 = int((y + height / 2) * image.shape[0])
        
        # Draw the bounding box on the image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Draw the label name on the image
        cv2.putText(image, label_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    # Display the image with the bounding boxes
    cv2.imshow('Image with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
# clear all variables  outputs in the jupyter notebook
from IPython import get_ipython
get_ipython().magic('reset -sf')



C:\Users\me1elar\AppData\Local\Temp\ipykernel_21428\3151676210.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')
